In [8]:
######################################
# Created by: Sardhendu Mishra
# With Regards to: Course CS 584
# CWID: A20388502
######################################


# import done here
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import itertools
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection

from IPython.display import display

### Loading the Dataset
---

In [9]:
# LOAD THE DATASET INTO PANDAS DATAFRAME:
datadir = '/Users/sam/All-Program/App-DataSet/Data-Science-Projects/Crime-Prediction/communities-crime-clean.csv'

crimeDF = pd.read_csv(datadir, sep=',', header='infer')

# Displaying first five elements for all the columns
print (crimeDF.shape)
with pd.option_context('display.max_columns', None):
    display(crimeDF.head())
    
# Important feature variables
nonPredictiveFeatures = ['state','communityname','fold']
regTargetVar = ['ViolentCrimesPerPop']
classTargetVar = ['highCrime']

(1993, 104)


,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR,HousVacant,PctHousOccup,PctHousOwnOcc,PctVacantBoarded,PctVacMore6Mos,MedYrHousBuilt,PctHousNoPhone,PctWOFullPlumb,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,RentLowQ,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,1,Alabastercity,7,0.01,0.61,0.21,0.83,0.02,0.01,0.41,0.49,0.26,0.21,0.02,1.0,0.46,0.77,0.23,0.38,0.22,0.16,0.22,0.42,0.29,0.29,0.27,0.16,0.24,0.38,0.26,0.01,0.14,0.27,0.37,0.39,0.12,0.69,0.27,0.37,0.32,0.48,0.31,0.20,0.35,0.34,0.56,0.81,0.79,0.97,0.82,0.60,0.51,0.00,0.09,0.00,0.50,0.43,0.46,0.43,0.04,0.03,0.03,0.02,0.97,0.01,0.17,0.21,0.65,0.64,0.56,0.82,0.10,0.22,0.5,0.01,0.85,0.82,0.00,0.26,0.83,0.13,0.66,0.16,0.15,0.13,0.13,0.23,0.25,0.28,0.19,0.18,0.25,0.00,0.00,0.03,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.0,0.06
1,1,AlexanderCitycity,10,0.01,0.41,0.55,0.57,0.01,0.00,0.47,0.45,0.31,0.57,0.00,0.0,0.15,0.33,0.13,0.27,0.65,0.41,0.45,0.21,0.22,0.29,0.15,0.00,0.29,0.00,0.14,0.02,0.45,0.57,0.70,0.21,0.27,0.43,1.00,0.17,0.93,0.24,0.42,0.32,0.54,0.52,0.46,0.48,0.43,0.48,0.59,0.81,0.82,0.02,0.42,0.00,0.00,0.78,0.64,0.54,0.00,0.02,0.01,0.01,0.98,0.02,0.19,0.18,0.43,0.43,0.42,0.58,0.17,0.46,0.5,0.02,0.73,0.58,0.18,0.40,0.56,0.55,0.48,0.06,0.06,0.08,0.02,0.07,0.08,0.09,0.20,0.16,0.36,0.00,0.00,0.00,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.0,0.14
2,1,Annistoncity,3,0.03,0.34,0.86,0.30,0.04,0.01,0.41,0.42,0.27,0.59,0.04,1.0,0.12,0.32,0.19,0.28,0.67,0.52,0.57,0.18,0.21,0.35,0.15,0.26,0.18,0.84,0.33,0.06,0.65,0.48,0.58,0.28,0.79,0.22,0.44,0.43,0.54,0.41,0.69,0.36,0.67,0.71,0.42,0.28,0.24,0.38,0.27,0.49,0.49,0.04,0.55,0.00,0.22,0.28,0.45,0.40,0.02,0.02,0.03,0.03,0.95,0.02,0.25,0.22,0.36,0.34,0.41,0.50,0.13,0.51,0.5,0.07,0.56,0.52,0.31,0.60,0.40,0.55,0.33,0.05,0.07,0.09,0.02,0.09,0.13,0.13,0.41,0.38,0.47,0.01,0.02,0.04,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.0,1.00
3,1,Athenscity,8,0.01,0.38,0.35,0.71,0.04,0.01,0.39,0.46,0.31,0.49,0.00,0.0,0.23,0.43,0.48,0.36,0.53,0.41,0.44,0.29,0.27,0.30,0.18,0.19,0.20,0.00,0.80,0.02,0.38,0.51,0.59,0.32,0.37,0.43,0.64,0.30,0.52,0.46,0.47,0.31,0.42,0.46,0.43,0.63,0.62,0.66,0.53,0.51,0.47,0.01,0.22,0.00,0.41,0.30,0.29,0.65,0.04,0.02,0.02,0.04,0.96,0.02,0.19,0.18,0.38,0.43,0.32,0.58,0.12,0.45,0.5,0.03,0.66,0.55,0.18,0.26,0.58,0.42,0.41,0.10,0.10,0.12,0.08,0.14,0.15,0.15,0.24,0.17,0.33,0.00,0.00,0.03,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.0,0.23
4,1,Auburncity,1,0.04,0.37,0.32,0.70,0.21,0.02,1.00,1.00,1.00,0.14,0.05,1.0,0.02,0.77,0.57,0.47,0.13,0.11,0.16,0.32,0.17,0.19,0.16,0.07,0.21,0.14,0.22,0.11,1.00,0.18,0.18,0.82,0.42,0.23,0.19,1.00,0.25,0.67,0.09,1.00,0.16,0.13,0.38,0.55,0.53,0.59,0.54,0.68,0.69,0.03,0.69,0.01,1.00,0.91,0.86,0.80,0.29,0.23,0.20,0.17,0.90,0.05,0.14,0.07,0.25,0.41,0.22,0.24,0.14,0.71,0.0,0.06,0.66,0.21,0.07,0.24,0.77,0.27,0.10,0.16,0.16,0.18,0.18,0.21,0.26,0.21,1.00,0.16,0.24,0.00,0.00,0.12,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.0,0.15


In [10]:
# The cross validation Random state to be used is for all the model is;
cv = model_selection.ShuffleSplit(n_splits=10,random_state=675) 

modelScores = defaultdict(list)
modelTopFeatures = defaultdict(list)

class ModelScoring():
    
    def __init__(self,modelName, modelObj):
        modelScores[modelName] = []
        modelTopFeatures[modelName] = []
        self.modelName = modelName
        self.model = modelObj
        self.cv = model_selection.ShuffleSplit(n_splits=10,
                                               random_state=675) 
        
    def setData(self, dataIN, targetIN):
        self.dataIN = dataIN
        self.targetIN = targetIN
        
    def cvEvaluate(self, scoringMetric, cv=None):
#         modelPerformaceMetric = OrderedDict()
        
        if cv:
            self.cv = cv
            
        for metric in scoringMetric:
            nFoldScore = model_selection.cross_val_score(self.model, 
                                                         self.dataIN, self.targetIN,
                                                         cv=self.cv, 
                                                         scoring=metric)

            modelScores[self.modelName].append({metric:np.mean(nFoldScore)})
            
    def evaluate(self, yTrue, yPred):
        metric = metrics.precision_recall_fscore_support(yTrue, yPred)
        modelScores[self.modelName].append({'precision':list(metric[0])})
        modelScores[self.modelName].append({'recall':list(metric[1])})
        modelScores[self.modelName].append({'fscore':list(metric[2])})
    
    def setTopFeatures(self, featureList, predictivePower):
        for feature, weight in zip(featureList,predictivePower):
            modelTopFeatures[self.modelName].append({feature:weight})
    
    
    def getTopFeatures(self, num=5, featureWeights=None):
        if not featureWeights:
            featureWeights = self.model.coef_
            
        feauturePredictivePower = pd.DataFrame(columns=['highCrimePredictive_features',
                                                      'high_featureWeight', 'lowCrimePredictive_features',
                                                      'low_featureWeight'])

        posTopWeight_index = featureWeights.flatten().argsort()[-num:][::-1]   # Find the indices of top most feature weights
        negTopWeight_index = featureWeights.flatten().argsort()[:num]

        posTopWeights = featureWeights.flatten()[posTopWeight_index]
        negTopWeights = featureWeights.flatten()[negTopWeight_index]

        high_CrimeFeatures = self.dataIN.columns[posTopWeight_index]
        low_CrimeFeatures = self.dataIN.columns[negTopWeight_index]

        feauturePredictivePower['highCrimePredictive_features'] = high_CrimeFeatures
        feauturePredictivePower['high_featureWeight'] = posTopWeights
        feauturePredictivePower['lowCrimePredictive_features'] = low_CrimeFeatures
        feauturePredictivePower['low_featureWeight'] = negTopWeights
#         print (high_CrimeFeatures+low_CrimeFeatures)
#         print (posTopWeights+negTopWeights)
        
        self.setTopFeatures(featureList = [item for item in itertools.chain(high_CrimeFeatures, low_CrimeFeatures)], 
                            predictivePower = [item for item in itertools.chain(posTopWeights, negTopWeights)])
        
        return feauturePredictivePower
    

# 1. Decision Trees:
----

### Part (a):  Decision Trees

#### Create a new field “highCrime” which is true if the crime rate per capita (ViolentCrimesPerPop) is greater than 0.1, and false otherwise. What are the percentage of positive and negative instances in the dataset?

Solutions: The percentage of positive instances is 62.72 and the percentage of negative insances is 37.28

In [11]:
crimeDF['highCrime'] = crimeDF['ViolentCrimesPerPop'] > 0.1
print ('The shape after adding the classification column is; ', crimeDF.shape)
percntgPositive = (sum(crimeDF['highCrime'] == True)/ (sum(crimeDF['highCrime'] == True) + sum(crimeDF['highCrime'] == False))) * 100
percntgNegative = 100-percntgPositive
print ('Percent of positive instancs are %f and percent of negative instances are %f '%(percntgPositive, percntgNegative))

The shape after adding the classification column is;  (1993, 105)
Percent of positive instancs are 62.719518 and percent of negative instances are 37.280482 


In [12]:
print (crimeDF.shape)
crimeDF.head()

(1993, 105)


,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop,highCrime
0,1,Alabastercity,7,0.01,0.61,0.21,0.83,0.02,0.01,0.41,...,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.0,0.06,False
1,1,AlexanderCitycity,10,0.01,0.41,0.55,0.57,0.01,0.00,0.47,...,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.0,0.14,True
2,1,Annistoncity,3,0.03,0.34,0.86,0.30,0.04,0.01,0.41,...,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.0,1.00,True
3,1,Athenscity,8,0.01,0.38,0.35,0.71,0.04,0.01,0.39,...,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.0,0.23,True
4,1,Auburncity,1,0.04,0.37,0.32,0.70,0.21,0.02,1.00,...,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.0,0.15,True


### Part (b):  Decision Trees

Data Preparation:

In [13]:
# converting the label into proper binary labels
lb = preprocessing.LabelBinarizer()

# Remove the non-predictive and the target column form the data
dataContinuous = crimeDF.drop(nonPredictiveFeatures+regTargetVar+classTargetVar, 1)  
# Convert the target column into binomial label vector
targetBinomial = lb.fit_transform(crimeDF['highCrime']).flatten()      

print ('The shape of dataForDT after preprocessing is: ', dataContinuous.shape)
print ('The shape of targetForDT after preprocessing is: ', targetBinomial.shape)
# print (dataForDT)
print (targetBinomial)
dataContinuous.head()

The shape of dataForDT after preprocessing is:  (1993, 100)
The shape of targetForDT after preprocessing is:  (1993,)
[0 1 1 ..., 1 1 0]


,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn
0,0.01,0.61,0.21,0.83,0.02,0.01,0.41,0.49,0.26,0.21,...,0.00,0.03,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.0
1,0.01,0.41,0.55,0.57,0.01,0.00,0.47,0.45,0.31,0.57,...,0.00,0.00,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.0
2,0.03,0.34,0.86,0.30,0.04,0.01,0.41,0.42,0.27,0.59,...,0.02,0.04,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.0
3,0.01,0.38,0.35,0.71,0.04,0.01,0.39,0.46,0.31,0.49,...,0.00,0.03,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.0
4,0.04,0.37,0.32,0.70,0.21,0.02,1.00,1.00,1.00,0.14,...,0.00,0.12,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.0


#### i. What are the training accuracy, precision, and recall for this tree? 

Solution: 

Accuracy = 1.0, Precision = 1.0, Recall = 1.0

In [14]:
# Fitting Decision tree classifier:
classifierDT = tree.DecisionTreeClassifier()
classifierDT = classifierDT.fit(dataContinuous, targetBinomial)

# Predict on the training Data
predForDT = classifierDT.predict(dataContinuous)

# ModelScoring
modelName = 'Decision Tree Classifier'
scoreObj = ModelScoring(modelName = modelName, modelObj=classifierDT)
scoreObj.evaluate(targetBinomial,predForDT)
modelScores[modelName]

[{'precision': [1.0, 1.0]}, {'recall': [1.0, 1.0]}, {'fscore': [1.0, 1.0]}]

In [15]:
# Calculate Precision and Recall
cnfMatrix = pd.crosstab(targetBinomial, predForDT)
cnfMatrix
tpDT =  cnfMatrix.iloc[1][1]
tnDT =  cnfMatrix.iloc[0][0]
fpDT =  cnfMatrix.iloc[0][1]
fnDT =  cnfMatrix.iloc[1][0]
print ('The accuracy of the decision tree classifier is: ', (tpDT+tnDT)/(tpDT+tnDT+fpDT+fnDT))
print ('The precision of the decision tree classifier is: ', tpDT/(tpDT+fpDT))
print ('The recall of the decision tree classifier is: ', tpDT/(tpDT+fnDT))
# cnfMatrix.iloc[0][0] + cnfMatrix.iloc[1][1]/(cnfMatrix.iloc[0][0] + cnfMatrix.iloc[1][1] + cnfMatrix.iloc[0][0] + cnfMatrix.iloc[1][1])
cnfMatrix

The accuracy of the decision tree classifier is:  1.0
The precision of the decision tree classifier is:  1.0
The recall of the decision tree classifier is:  1.0


col_0,0,1
row_0,,
0,743,0
1,0,1250


#### ii. What are the main features used for classification? Can you explain why they make sense (or not)?

Solution: The top five features for classification are:

PctKids2Par : Percentage of kids in family housing with two parents : This feature makes sense as a negative correlation to the dependent variable because a perpetrator would be reluctant on commiting violent crimes in front of a child. The more the house is of childrens the less would be the proximity of violent crimes.

racePctWhite : percentage of population that is caucasian: This feature makes sense again as a negative correlation to the dependent variable because its evedent that the more the percentage of people are white the less the crime is. Evidently more proportion of white people are employed and educated, and in general educated and emlployed people have very less probability of commiting violent crime.

PctLess9thGrade: percentage of people 25 and over with less than a 9th grade education: People with less than 9th grade and with a age of more than 25% sould incomparison engage in more crime/violent crime because education is directly propotional to employment and people nature and attitude. Employed people with genial nature would seldom engage in crime.

PctEmplManu : percentage of people 16 and over who are employed in manufacturing

blackPerCap : per capita income for african americans: This variable would again have a negative correlationship as stated above employed people with a descent salary would seldom engage in violent crime.

In [16]:
# Get the best Fetures: Information gain with GINI
ImpfeatureDF = pd.DataFrame(columns=['feature_name', 'infoGain'])
featureVec = np.array(dataContinuous.columns)
ImpfeatureDF['feature_name'] = np.array(dataContinuous.columns)
# print (featureVec)
ImpfeatureDF['infoGain'] = classifierDT.feature_importances_

print ('The Top 5 features selected by decision tree are: ')
inpFeatue = ImpfeatureDF.sort_values('infoGain', ascending=False).head(10)

scoreObj.setTopFeatures(inpFeatue['feature_name'], inpFeatue['infoGain'])
inpFeatue

The Top 5 features selected by decision tree are: 


,feature_name,infoGain
44,PctKids2Par,0.360083
3,racePctWhite,0.088177
5,racePctHisp,0.047067
29,PctLess9thGrade,0.020824
34,PctEmplManu,0.017148
73,PctHousOwnOcc,0.014166
71,HousVacant,0.013431
26,HispPerCap,0.013414
85,MedRent,0.013368
24,AsianPerCap,0.013347


### Part (c): Decision Trees

#### i. What are the 10-fold cross-validation accuracy, precision, and recall?

Solution: mean 10-fold Accuracy = 0.7755, mean 10-fold Precision = 0.8016 and mean 10-fold Recall = 0.83343

In [17]:
# import random
# random.seed(3213)
classifierDT = tree.DecisionTreeClassifier()

modelName = 'Decision Tree cv Classifier'
scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierDT)
scoreOBJ.setData(dataIN=dataContinuous, targetIN=targetBinomial)
scoreOBJ.cvEvaluate(scoringMetric=['accuracy', 'precision', 'recall'])

modelScores[modelName]

[{'accuracy': 0.77299999999999991},
 {'precision': 0.80138770150926086},
 {'recall': 0.82182553257930235}]

#### ii. Why are they different from the results in the previous test?

Solution:
In the first Decision Tree fit, since we fit and classify on the whole dataset, the tree learned the entire data and made very good classification with 100% accuracy, precision and recall. This states that the previous method is prone to overfitting and may not generalize well for a new data, as data in the real world are always noisy. In the current model since we do crossvalidation we train on separate data and test on separate data. The model performs poorly on the test data and hence the accuracy. precision and recall of the model decreases.

# 2. Linear Classification:
-----

### Part (a) :-  GaussianNB

#### i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

Solutions: mean n-fold Accuracy = 0.7835, mean n-fold Precision = 0.9312 and mean Recall = 0.69

In [18]:
from sklearn.naive_bayes import GaussianNB
classifierGNB = GaussianNB()
# classifierGNB.fit()

modelName = 'Gaussian Naive Bayes Classifier'
scoreGNB = ModelScoring(modelName = modelName, modelObj = classifierGNB)
scoreGNB.setData(dataIN=dataContinuous, targetIN=targetBinomial)
scoreGNB.cvEvaluate(scoringMetric=['accuracy', 'precision', 'recall'])

modelScores[modelName]

[{'accuracy': 0.78350000000000009},
 {'precision': 0.93122871464730816},
 {'recall': 0.69799123705957622}]

#### ii. What are the 10 most predictive features? This can be measured by the normalized absolute difference of means for the feature between the two classes:

Solution: The 10 most predictive features are given below.


PctKids2Par: As discussed above this feature has a negative weight which means that it is mostely negative correlated to the dependent variable. The more the house is of childrens the less would be the proximity of violent crimes. As perpetratos would be more reluctant in commiting violent crime in a house full of childrens.

PctFam2Par: percentage of families (with kids) that are headed by two parents

racePctWhite : This feature has a negative weight which indicates a negative correlation with the dependent variable. Evidently more proportion of white people are employed and educated, and in general educated and emlployed people have very less probability of commiting violent crime.

PctIlleg: percentage of kids born to never married; 

FemalePctDiv: percentage of females who are divorced

TotalPctDiv

PctYoungKids2Par

pctWInvInc

PctTeen2Par

MalePctDivorce


---> The larger this different, the more predictive the feature. Why do these make sense?
This makes sense because each feature would have a mean value for corresponding to each class. If the difference between the mean is big, this indicated that the feature provides a very good seperation between the different class. The denominator indicates a lower standard deviation is proportional to higher predictive power. This makes sense because if the standard deviation is large, the gaussian distribution would result in a shorter and wider curve. For a particular feature, this indicates that the data is widely distributed and there is high chance that the two distribution corresponding to two class might intersect, which may result in bad separation between the different clss.

In [19]:
indexClassNo = np.where(targetBinomial==0)[0]
indexClassYes = np.where(targetBinomial==1)[0]
# print (indexClassYes)

NBfeauturePredictivePower = pd.DataFrame(columns=['feature_name','PredictivePower','NegPos-Coefficient'])
NBfeauturePredictivePower['feature_name'] = np.array(dataContinuous.columns)

for num, features in enumerate(dataContinuous.columns):
#     print (features)
    meanNo = np.mean(dataContinuous[features][indexClassNo])
    meanYes = np.mean(dataContinuous[features][indexClassYes])
    stdNo = np.std(dataContinuous[features][indexClassNo])
    stdYes = np.std(dataContinuous[features][indexClassYes])
    
    predictivePower = np.abs(meanYes - meanNo)/(stdYes + stdNo)
    negPosCoefficient = (meanYes - meanNo)/(stdYes + stdNo)
    NBfeauturePredictivePower.ix[num, 'PredictivePower'] = predictivePower
    NBfeauturePredictivePower.ix[num, 'NegPos-Coefficient'] = negPosCoefficient


topFeature = NBfeauturePredictivePower.sort_values('PredictivePower', ascending=False).head(10)
scoreGNB.setTopFeatures(topFeature['feature_name'], topFeature['NegPos-Coefficient'])

topFeature

,feature_name,PredictivePower,NegPos-Coefficient
44,PctKids2Par,0.809748,-0.809748
43,PctFam2Par,0.745545,-0.745545
3,racePctWhite,0.73523,-0.73523
50,PctIlleg,0.709261,0.709261
40,FemalePctDiv,0.693978,0.693978
41,TotalPctDiv,0.674645,0.674645
45,PctYoungKids2Par,0.665009,-0.665009
15,pctWInvInc,0.661076,-0.661076
46,PctTeen2Par,0.642949,-0.642949
38,MalePctDivorce,0.616864,0.616864


#### iii. How do these results compare with your results from decision trees, above?

Sol: Feature "PctKids2Par" is treated as the most significant feature by both the algorithms. Also, "racePctWhite" is treated important by both the algorithms. All other features are different in both the cases. The difference is majorly because decision tree choses fits fetures on a forward selection basis and in doing so its choice for the next feature highly depends on the previously chosen feature, which introduces a sense of correlation between the features. In Gaussian Naive bayes the features are trated independent given the class. 

For example seeing the top 10 in both the case. 

For a decision tree case "PctEmploy", "PctLess9thGrade", "PctEmplManu", can be thought of highly correlated to each other because they talk about education or emloyment. Also features "PctWOFullPlumb", "HousVacant", "PctVacMore6Mos" are also significantly correlated as houses with less plumbing would be less occupied and so on. However, in the case of Gaussian Naive Bayes the features are seldom related, It seems more random than the feature selection in Decision Tree.

### Part (b) :-  LinearSVC

#### i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

Solutions: mean n-fold Accuracy = 0.839, mean n-fold Precision = 0.864348097316 and mean Recall = 0.87395706326

In [20]:
from sklearn.svm import LinearSVC
classifierSVC = LinearSVC()
# classifierGNB.fit()

modelName = 'Linear SVC Classifier'
scoreLSVC = ModelScoring(modelName = modelName, modelObj = classifierSVC)
scoreLSVC.setData(dataIN=dataContinuous, targetIN=targetBinomial)
scoreLSVC.cvEvaluate(scoringMetric=['accuracy', 'precision', 'recall'])

modelScores[modelName]

[{'accuracy': 0.83900000000000008},
 {'precision': 0.86434809731621909},
 {'recall': 0.8739570632604845}]

#### ii. What are the 10 most predictive features? This can be measured by the absolute feature weights in the model. Why do these make sense (or not)?

In [21]:
classifierSVC.fit(dataContinuous, targetBinomial)

SVCfeauturePredictivePower = pd.DataFrame(columns=['feature_name','absFeatureWeights', 'featureWeights'])
SVCfeauturePredictivePower['feature_name'] = np.array(dataContinuous.columns)
SVCfeauturePredictivePower['absFeatureWeights'] = np.abs(classifierSVC.coef_).flatten()
SVCfeauturePredictivePower['featureWeights'] = classifierSVC.coef_.flatten()
topFeatures = SVCfeauturePredictivePower.sort_values('absFeatureWeights', ascending=False).head(10)

scoreLSVC.setTopFeatures(topFeatures['feature_name'], topFeatures['featureWeights'])

topFeatures

,feature_name,absFeatureWeights,featureWeights
15,pctWInvInc,1.888487,-1.888487
64,PersPerOccupHous,1.755123,1.755123
3,racePctWhite,1.500220,-1.500220
44,PctKids2Par,1.190328,-1.190328
84,RentHighQ,1.066884,1.066884
38,MalePctDivorce,1.065696,1.065696
27,NumUnderPov,1.051548,1.051548
90,NumStreet,1.019155,1.019155
37,PctOccupMgmtProf,1.014675,1.014675
0,population,1.002301,1.002301


#### iii. How do these results compare with your results from decision trees, above

Sol: The feature selection in linear SVC is very different from decision trees in the sense that feature selection in decision tree depends higly on the first feature selected.

Only "racePctWhite" managed to get into the top 10 features for both the algorithms. however the feature selection in linearSVC has many interesting features such as "Population", "PersPerOccupHours", and "RentHighQ".

# 3. Regression:
-------

## Part (a) : Linear regression

#### i. Using 10-fold cross-validation, what is the estimated mean-squared-error (MSE) of the model?

Sol: Estimated MSE 10-fold: 0.0189222775959

In [22]:
# Load the data to fit linear model, To fit the linear model the data would be the same, however the target would change to continuous Variable
# Remove the non-predictive and the target column form the data
targetContinuous = crimeDF[regTargetVar]

# Fit a linear model for cross vlidation:
from sklearn.linear_model import LinearRegression

classifierLR = LinearRegression()

modelName='Linear Regression Classifier'
scoreLR = ModelScoring(modelName = modelName, modelObj = classifierLR)
scoreLR.setData(dataIN=dataContinuous, targetIN=targetContinuous)
scoreLR.cvEvaluate(scoringMetric=['neg_mean_squared_error'])

modelScores[modelName]


[{'neg_mean_squared_error': -0.018922277595922486}]

#### ii. What is the MSE on the training set (train on all the data then test on it all)?

Sol: 0.0165167748803

In [23]:
classifierLR.fit(dataContinuous, targetContinuous)
predictContinuousLR = classifierLR.predict(dataContinuous).flatten()
mseLR = metrics.mean_squared_error(targetContinuous, predictContinuousLR)

print ('The MSE on the training set for the Linear model is :', mseLR)

The MSE on the training set for the Linear model is : 0.0165167748803


#### iii. What features are most predictive of a high crime rate? A low crime rate?

Sol: 

HighCrime : PersPerOccupation, PctHousownOcc, MalePctDivorce, PctRecImmig8, MedRent

LowCrime: PctPersOwnOccup, TotalPctDiv, whitePerCap, PctKids2Par, OwnOccLowQuart

In [24]:
topFeatureDF = scoreLR.getTopFeatures(num=5)
topFeatureDF.head()

,highCrimePredictive_features,high_featureWeight,lowCrimePredictive_features,low_featureWeight
0,PersPerOccupHous,0.635088,PctPersOwnOccup,-0.675694
1,PctHousOwnOcc,0.568133,TotalPctDiv,-0.561924
2,MalePctDivorce,0.458517,whitePerCap,-0.351016
3,PctRecImmig8,0.432511,PctKids2Par,-0.322651
4,MedRent,0.372728,OwnOccLowQuart,-0.308170


## Part (b) : Ridge Regression 

#### i. What is the estimated MSE of the model under 10-fold CV?

Sol: 0.01858

In [26]:
from sklearn.linear_model import RidgeCV
alpha = np.array([10, 1, 0.1, 0.01, 0.001], dtype='float64')

classifierRR = RidgeCV(alpha)

modelName='Ridge Regression Classifier'
scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierRR)
scoreOBJ.setData(dataIN=dataContinuous, targetIN=targetContinuous)
scoreOBJ.cvEvaluate(scoringMetric=['neg_mean_squared_error'])

modelScores[modelName]

[{'neg_mean_squared_error': -0.018581731506130121}]

#### ii. What is the MSE on the training set (train on all the data then test on it all)?

Sol: 0.0167635291552

In [27]:
from sklearn.linear_model import Ridge

# classifierRR = Ridge()
classifierRR = classifierRR.fit(dataContinuous, targetContinuous)
predictionContinuousRR = classifierRR.predict(dataContinuous)

mseRR = metrics.mean_squared_error(targetContinuous, predictionContinuousRR)
print ('The MSE for the Ridge Regression model with default parameter setting is: ', mseRR)


The MSE for the Ridge Regression model with default parameter setting is:  0.0167635291552


#### iii. What is the best alpha?

Sol: The best alpha value is 1

In [28]:
classifierRR.fit(dataContinuous, targetContinuous)
classifierRR.alpha_

1.0

#### iv. What does this say about the amount of overfitting in linear regression for this problem?

Solution: The alpha value of 1 is found to be the best alpha using Cross Validation Score. This means we are neither enforcing strong regularization nor are be enforcing strong likelihood. But we are still penalizing the model using L2 norm. This also means that the model with only the likelihood is not very highly prone to overfitting but is still a little bit prone to overfitting, since we are penalizing the likelihood by weighting the regularization term by alpha=1.


## Part (c) : Polynomial Regression

#### i. What is the estimated MSE of the model under 10-fold CV?

The mean MSE for 10-fold cross validation (Polynomial Regression with L2 norm) is 0.017610408756280137

The mean MSE for 10-fold cross validation (Polynomial Regression) is 0.100727

In [29]:
from sklearn.preprocessing import PolynomialFeatures

# Get the Polynomial Features Data
classifierPR = PolynomialFeatures(degree=2, include_bias=False)
dataContinuousPoly = classifierPR.fit_transform(dataContinuous)
print ('The shape of the 2nd order dataset is: ', dataContinuousPoly.shape)

classifierLR_poly = LinearRegression()
classifierRR_Poly = RidgeCV(alpha)

modelName='Polynomial Ridge Regression Classifier'
scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierRR_Poly)
scoreOBJ.setData(dataIN=dataContinuousPoly, targetIN=targetContinuous)
scoreOBJ.cvEvaluate(scoringMetric=['neg_mean_squared_error'])

modelScores[modelName]

The shape of the 2nd order dataset is:  (1993, 5150)


[{'neg_mean_squared_error': -0.017610408756280137}]

#### ii. What is the MSE on the training set (train on all the data then test on it all)?

Sol: The mean squared error (polynomial features) of the linear regression model is 1.09352811987e-28 

In [30]:
classifierLR_poly = classifierLR_poly.fit(dataContinuousPoly, targetContinuous)
predictContinuousLR_poly = classifierLR_poly.predict(dataContinuousPoly).flatten()

mseLR_poly = metrics.mean_squared_error(targetContinuous, predictContinuousLR_poly)

print ('The mean squared error (polynomial features) of the linear regression model is %s : '%(str(mseLR_poly)))

The mean squared error (polynomial features) of the linear regression model is 1.09352811987e-28 : 


#### iii. Does this mean the quadratic model is better than the linear model for this problem?

Sol: Despite the MSE of the polynomial feature space (2nd order) is much closer to 0, the quadratic model is not a good fit. Because, since we train and test on the same dataset, the model learns the training data very well. This scenario may lead to overfitting as the real world dataset is more noisy. 

# Dirty Data
------

### Part (a):  Repeat the decision tree learning question for the full (non-clean) data set and present the results. Are the CV results better or worse? What does this say about the effect of missing values?

Solution: 

Clean Data: mean 10-fold Accuracy = 0.7735, mean 10-fold Precision = 0.802457540816 and mean 10-fold Recall = 0.827406078072

Dirty Data: mean 10-fold Accuracy = 0.752, mean 10-fold Precision = 0.8067646146 and mean 10-fold Recall = 0.803254339078.

Comparing the performaces of the decision tree modeled on the dirty data, we see that the accuracy and recall decreases by approximately by 2% and the precision increases by 0.04%, all which are not a very big difference. Since we add a lot of features in the dirty data, we increase the dimensionality which affect the model performance (the curse of dimensionality) and hence the model is more prone to overfit. Hence the slight differnce in the model with the dirty data. 

Analysis: We find the distribution of the violentCrime columns to all the columns that have missing value, The distribution table can be seen below. As we can see that the distribution is exactely the same for all the column with missing value, this cannot be just by chance. Hence we can say that the values are missing not at random.

Also, the distribution is (violentCrimes=0.573731, nonViolent=crimes=0.426269)  which is not very far from the actual distibution of the label column which is (violentCrimes=62.72 and nonViolent-crimes=37.28). This is also a reason that we dont see a lot of change in the two decision tree model build with clean data and dirty data.

In [31]:
datadir = '/Users/sam/All-Program/App-DataSet/Data-Science-Projects/Crime-Prediction/communities-crime-full.csv'

crimeDF_dirty = pd.read_csv(datadir, header='infer', sep=',')

print (crimeDF_dirty.shape)
with pd.option_context('display.max_columns', None):
    display(crimeDF_dirty.head())

(1994, 128)


,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR,HousVacant,PctHousOccup,PctHousOwnOcc,PctVacantBoarded,PctVacMore6Mos,MedYrHousBuilt,PctHousNoPhone,PctWOFullPlumb,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,RentLowQ,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,PctPolicWhite,PctPolicBlack,PctPolicHisp,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,0.20,1.0,0.37,0.72,0.34,0.60,0.29,0.15,0.43,0.39,0.40,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.10,0.18,0.48,0.27,0.68,0.23,0.41,0.25,0.52,0.68,0.40,0.75,0.75,0.35,0.55,0.59,0.61,0.56,0.74,0.76,0.04,0.14,0.03,0.24,0.27,0.37,0.39,0.07,0.07,0.08,0.08,0.89,0.06,0.14,0.13,0.33,0.39,0.28,0.55,0.09,0.51,0.5,0.21,0.71,0.52,0.05,0.26,0.65,0.14,0.06,0.22,0.19,0.18,0.36,0.35,0.38,0.34,0.38,0.46,0.25,0.04,0.0,0.12,0.42,0.50,0.51,0.64,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,0.94,0.93,0.03,0.07,0.1,0.07,0.02,0.57,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.30,0.22,0.35,0.01,0.24,0.14,0.24,0.30,0.27,0.73,0.57,0.15,0.42,0.36,1.00,0.63,0.91,1.00,0.29,0.43,0.47,0.60,0.39,0.46,0.53,0.00,0.24,0.01,0.52,0.62,0.64,0.63,0.25,0.27,0.25,0.23,0.84,0.10,0.16,0.10,0.17,0.29,0.17,0.26,0.20,0.82,0.0,0.02,0.79,0.24,0.02,0.25,0.65,0.16,0.00,0.21,0.20,0.21,0.42,0.38,0.40,0.37,0.29,0.32,0.18,0.00,0.0,0.21,0.50,0.34,0.60,0.52,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.00,0.0,0.30,0.58,0.19,0.39,0.38,0.40,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,0.29,0.49,0.32,0.63,0.41,0.71,0.70,0.45,0.42,0.44,0.43,0.43,0.71,0.67,0.01,0.46,0.00,0.07,0.06,0.15,0.19,0.02,0.02,0.04,0.05,0.88,0.04,0.20,0.20,0.46,0.52,0.43,0.42,0.15,0.51,0.5,0.01,0.86,0.41,0.29,0.30,0.52,0.47,0.45,0.18,0.17,0.16,0.27,0.29,0.27,0.31,0.48,0.39,0.28,0.00,0.0,0.14,0.49,0.54,0.67,0.56,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.20,0.82,0.51,0.36,0.40,0.39,0.16,0.25,0.36,0.44,0.01,0.10,0.09,0.25,0.31,0.33,0.71,0.36,0.45,0.37,0.39,0.34,0.45,0.49,0.44,0.75,0.65,0.54,0.83,0.65,0.85,0.86,0.03,0.33,0.02,0.11,0.20,0.30,0.31,0.05,0.08,0.11,0.11,0.81,0.08,0.56,0.62,0.85,0.77,1.00,0.94,0.12,0.01,0.5,0.01,0.97,0.96,0.60,0.47,0.52,0.11,0.11,0.24,0.21,0.19,0.75,0.70,0.77,0.89,0.63,0.51,0.47,0.0

--> Data Preparation

In [32]:
nonPredictiveFeatures = ['state','county','community', 'communityname','fold']
regTargetVar = ['ViolentCrimesPerPop']
classTargetVar = ['highCrime']

# Convert the lebel form continuous to discrete
crimeDF_dirty['highCrime'] = crimeDF_dirty['ViolentCrimesPerPop'] > 0.1

crimeDF_dirty.head()

# converting the label into proper binary labels
lb = preprocessing.LabelBinarizer()

# # Remove the non-predictive and the target column form the data
dataContinuous_dirty = crimeDF_dirty.drop(nonPredictiveFeatures+regTargetVar+classTargetVar, 1)  
# # Convert the target column into binomial label vector
targetBinomial_dirty = lb.fit_transform(crimeDF_dirty['highCrime']).flatten()      

print ('The shape of dataForDT after preprocessing is: ', dataContinuous_dirty.shape)
print ('The shape of targetForDT after preprocessing is: ', targetBinomial_dirty.shape)
# print (dataForDT)
print (targetBinomial_dirty)
dataContinuous_dirty.head()

The shape of dataForDT after preprocessing is:  (1994, 122)
The shape of targetForDT after preprocessing is:  (1994,)
[1 1 1 ..., 1 1 1]


,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,?,0.02,0.12,0.45,?,?,?,?,0.00,?
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,?,0.01,0.21,0.02,?,?,?,?,0.00,?
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,?,0.02,0.39,0.28,?,?,?,?,0.00,?
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,?,0.04,0.09,0.02,?,?,?,?,0.00,?


--> Imputing missing values with the mean

In [33]:
# We inpute missing value with the mean because there is a 
# high probability of data having a value of 0 
# so imputing with zero would bais the decision tree model
# Also, there is very less probability for the data
# to have the value of exact mean at float64.
# Also, For decision tree classification, since we are splitting,
# imputing any value should not make a lot of difference

dataContinuous_dirtyMean = dataContinuous_dirty

# First We replace all the ? with np.NaN
dataContinuous_dirtyMean = dataContinuous_dirtyMean.replace(to_replace='?', value=np.NaN)
dataContinuous_dirtyMean = dataContinuous_dirtyMean.astype('float64')

meanDF = dataContinuous_dirtyMean.mean(axis=0, skipna=True)

def inputeMean(dataFrameIN):
    for columns in dataFrameIN.columns:
        dataFrameIN[columns] = dataFrameIN[columns].replace(to_replace=np.NaN, value=meanDF[columns])
    return (dataFrameIN)
    
dataContinuous_dirtyMean = inputeMean(dataContinuous_dirtyMean)

dataContinuous_dirtyMean.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.290000,0.12,0.26,0.20,0.060000,0.040000,0.900000,0.500000,0.32,0.140000
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.305987,0.02,0.12,0.45,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.305987,0.01,0.21,0.02,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.305987,0.02,0.39,0.28,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.305987,0.04,0.09,0.02,0.163103,0.076708,0.698589,0.440439,0.00,0.195078


--> Fit the Decision tree model

In [34]:
# Fit the Decision Tree Model
classifierDT_dirty = tree.DecisionTreeClassifier()


modelName='DirtyData: Decision Tree Classifier'
scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierDT_dirty)
scoreOBJ.setData(dataIN=dataContinuous_dirtyMean, targetIN=targetBinomial_dirty)
scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

modelScores[modelName]

[{'accuracy': 0.75600000000000001},
 {'precision': 0.8109243885090367},
 {'recall': 0.80587961948526821}]

--> Find the distribution of the class variable given few column with missing value (the column data in grouped by on missing value "?")

In [35]:
def checkMissingDist(dataFrameIN, columnNameList):
    distMissing_DF = pd.DataFrame(columns=['%non-ViolentCrime', '%ViolentCrime'])
    for columnName in columnNameList:
        dfNew = pd.DataFrame(columns=[columnName, 'target'])
        dfNew[columnName] = dataFrameIN[columnName]
        dfNew['target'] = targetBinomial_dirty
        dfcrosstab = dfNew[(dfNew[columnName] == '?')].groupby('target').size()
#         print (dfcrosstab[0]/(dfcrosstab[0]+dfcrosstab[1]))
#         print (dfcrosstab[1]/(dfcrosstab[0]+dfcrosstab[1]))
        distMissing_DF.ix[columnName,'%non-ViolentCrime'] = dfcrosstab[0]/(dfcrosstab[0]+dfcrosstab[1])
        distMissing_DF.ix[columnName,'%ViolentCrime'] = dfcrosstab[1]/(dfcrosstab[0]+dfcrosstab[1])
    return distMissing_DF
    
columnNameList = ['LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'PolicOperBudg', 'PolicCars', 'PolicAveOTWorked', 'PolicBudgPerPop']
distMissing_DF = checkMissingDist(dataContinuous_dirty, columnNameList)
                                  
distMissing_DF

,%non-ViolentCrime,%ViolentCrime
LemasPctPolicOnPatr,0.426269,0.573731
LemasGangUnitDeploy,0.426269,0.573731
PolicOperBudg,0.426269,0.573731
PolicCars,0.426269,0.573731
PolicAveOTWorked,0.426269,0.573731
PolicBudgPerPop,0.426269,0.573731


--> Imputing missing values with zero (0)

In [28]:
# dataContinuous_dirty0 = dataContinuous_dirty

# # We replace all the ? with 0
# dataContinuous_dirty0 = dataContinuous_dirty0.replace(to_replace='?', value=0)

# # We convert he datatypes into float64
# dataContinuous_dirty0 = dataContinuous_dirty0.astype('float64')

# # Decision Tree Classifier
# classifierDT_dirty = tree.DecisionTreeClassifier()

# # Performance Metric
# nfoldACC = model_selection.cross_val_score(classifierDT_dirty, dataContinuous_dirty0, targetBinomial_dirty, cv=cv, scoring='accuracy')
# nfoldPrecision = model_selection.cross_val_score(classifierDT_dirty, dataContinuous_dirty0, targetBinomial_dirty, cv=cv, scoring='precision')
# nfoldRecall = model_selection.cross_val_score(classifierDT_dirty, dataContinuous_dirty0, targetBinomial_dirty, cv=cv, scoring='recall')

# print ('The 10 Fold accuracy of the decision tree classifier is %s and their mean is : '%(str(nfoldACC)), np.mean(nfoldACC))
# print ('The 10 Fold precision of the decision tree classifier is %s and their mean is : '%(str(nfoldPrecision)), np.mean(nfoldPrecision))
# print ('The 10 Fold recall of the decision tree classifier is %s and their mean is : '%(str(nfoldRecall)), np.mean(nfoldRecall))

# Extra Credits
-------

## Part (a) :- Do a team requirement (above) that your team is not already required to do.

Step:

Step 1: We replace all the missing values of the dirty data with the mean. We also experiment with median, but the feature selection technique doesnt choose any of the features with missing data.

Step 2: We use Lasso Feature selection technique to find the most predictive features in the dataset (both clean and dirty). Lasso finds 34 most important features out of the 100 features from the clean dataset and 37 features out of 122 features from the Dirty Data.

Step 3: To the new data (clean and dirty) consisting only the features selected, we fit a simple Logistic Regression model and compute the precision, recall and accuracy using CrossValidation.

Step 4: To the new data (clean and dirty) consisting only the features selected, we fit a SVM model usign a polynomial kernel (3rd order) and compute the precision, recall and accuracy using CrossValidation.

Note: Lasso Regression and Logistic Regression with L1 norm produces different outputs and performance mesure. While Evaluating models we see that the features selection with Lasso using Logistic (with L1 norm) was better than simple Logistic with L1 norm. That is the reason we do LASSO feature selection separately.

#### 1) Experiment with two learning methods other than those described above (one can be a non-linear kernel for SVM) for the classification problem, explaining clearly what you did. Show CV results for both the clean and full datasets.

Cross Validation Results:

Clean Data; Model (Lasso Feature selection; Logistic with L1 Norm) Performance Metric: [{'accuracy': 0.85149999999999992}, {'precision': 0.8709122026050311}, {'recall': 0.88880138359548133}]

Clean Data: Model (Lasso Feature selection; Poly SVM) : Performance Metric: [{'accuracy': 0.83050000000000002}, {'precision': 0.84073677346707765}, {'recall': 0.89131720925893521}]

Dirty Data: Model (Lasso Feature selection; Logistic with L1 Norm) : Performance Metric: [{'accuracy': 0.84999999999999998}, {'precision': 0.88315316441279157}, {'recall': 0.88196204423858404}]

Dirty Data: Model (Lasso Feature selection; Poly SVM) : Performance Metric: [{'accuracy': 0.83699999999999997}, {'precision': 0.86248894734647141}, {'recall': 0.88574373106459969}]


#### ii) What method gives the best results?

Solution: Logistic regression on feature selection with LASSO gives the best Result. Because the cross validation accuracy, precision and recall are  better than the polynomial SVM of 3rd degree. Additionally, the model size in the simple Logistic regression is only 34 which is very very small compared to the 3rd order polynomial feature space of SVM. This satisfies Ocam Razor that states that the smaller model that provides better performance cannot be by chance. 

The evaluation score of models with dirty data is very close to the evaluation score of models with clean data. This is because the LASSO feature selection technique removes all the irrelevant featues from the dirty data. This also states that the features with missing values in the dirty data are not relevant and does not improve the model performance.

#### iii) What feature(s) seem to be most consistently predictive of high crime rates? How reliable is this conclusion?

TotalPctDiv: Has +ve predictive coefficient for classification probelems (Gaussian Naive bayes) but negative cofficient for regression problems.

racePctHisp: Was said to have high information gain for decision trees

population was also said to be a +ve predictive feature for Linear SVC

RentHighQ was said to have +ve predictive power for linear SVC

FemalePctDiv was said to have +ve predictive power for Gaisian Naive Bayes

PctTeen2Par +ve predictive power for gaussian naive bayes

The conclusion are reliable because the model are evaluated using 10 fold cross validation. The model (Lasso features with Logistic) produces the best results when compared to all other models only by using 34 features. Most of the features indicating low crimes and some of the features high crimes that are selected were also selected by other models such as Linear SVC, Gaussian naive bayes and others. This also states that the featues selection is proper.

--> Clean Data: Lasso Feature Selection

In [36]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

print ('The shape of Clean Data is: ', dataContinuous.shape)

# Fit LASSO (Linear regression with L1 Norm)
classifierLS_clean = linear_model.Lasso(alpha=0.001, 
                                  fit_intercept=True, 
                                  max_iter=1000)

classifierLS_clean.fit(dataContinuous, targetBinomial)

print ('CleanData: Number of Features selected by Lasso is: ', len(np.where(classifierLS_clean.coef_ != 0)[0]))



modelName='Clean Data: Lasso Features'
clnLasso_scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierLS_clean)
clnLasso_scoreOBJ.setData(dataIN=dataContinuous, targetIN=targetBinomial)
topFeatureDF = clnLasso_scoreOBJ.getTopFeatures(num=5)
print ('The first degree features most predictive of a high crime rate and low crime rate and their respective weights are: ')
topFeatureDF.head(5)

The shape of Clean Data is:  (1993, 100)
CleanData: Number of Features selected by Lasso is:  34
The first degree features most predictive of a high crime rate and low crime rate and their respective weights are: 


,highCrimePredictive_features,high_featureWeight,lowCrimePredictive_features,low_featureWeight
0,TotalPctDiv,0.351936,pctWInvInc,-0.765809
1,racePctHisp,0.217541,PctKids2Par,-0.412368
2,OwnOccHiQuart,0.114368,PersPerOwnOccHous,-0.242417
3,OtherPerCap,0.099266,racePctWhite,-0.227684
4,population,0.084624,NumIlleg,-0.152096


--> Dirty Data: Lasso Feature Selection

In [37]:
print ('The shape of Clean Data is: ', dataContinuous_dirtyMean.shape)

classifierLS_dirty = linear_model.Lasso(alpha=0.001, 
                                  fit_intercept=True, 
                                  max_iter=1000)

classifierLS_dirty.fit(dataContinuous_dirtyMean, targetBinomial_dirty)

print ('CleanData: Number of Features selected by Lasso is: ', len(np.where(classifierLS_dirty.coef_ != 0)[0]))



modelName='Dirty Data: Lasso Features'
drtyLasso_scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierLS_dirty)
drtyLasso_scoreOBJ.setData(dataIN=dataContinuous_dirtyMean, targetIN=targetBinomial_dirty)
topFeatureDF = drtyLasso_scoreOBJ.getTopFeatures(num=5)
print ('The first degree features most predictive of a high crime rate and low crime rate and their respective weights are: ')
topFeatureDF.head(5)

The shape of Clean Data is:  (1994, 122)
CleanData: Number of Features selected by Lasso is:  37
The first degree features most predictive of a high crime rate and low crime rate and their respective weights are: 


,highCrimePredictive_features,high_featureWeight,lowCrimePredictive_features,low_featureWeight
0,TotalPctDiv,0.374450,pctWInvInc,-0.758303
1,RacialMatchCommPol,0.288589,PctKids2Par,-0.405172
2,racePctHisp,0.201255,racePctWhite,-0.287850
3,OwnOccHiQuart,0.118270,PersPerOwnOccHous,-0.217569
4,OtherPerCap,0.101677,PctPolicMinor,-0.183767


--> Logistic Regresion with L1 norm: Clean Data and Dirty Data:

In [39]:
# Initialize the Logistic model with L1 norm:
C = 1
classifierLR_L1_poly = linear_model.LogisticRegression(penalty='l1', 
                                                       C=C, fit_intercept=True, 
                                                       max_iter=1000)

####### For Clean Data
newCoeff = classifierLS_clean.coef_.flatten()
newCleanData = dataContinuous[dataContinuous.columns[np.where(newCoeff!=0)[0]]]
print ('Clean Data: The Shape of New data is: ', newCleanData.shape)

modelName='Clean Data; Lasso Feature selection; Logistic with L1 Norm'
cleanLasso_scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierLR_L1_poly)
cleanLasso_scoreOBJ.setData(dataIN=newCleanData, targetIN=targetBinomial)
cleanLasso_scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

print ('Model (%s) : Performance Metric: \n'%modelName, modelScores[modelName])
print ('')



#######  For Dirty Data
newCoeff = classifierLS_dirty.coef_.flatten()
newDirtyData = dataContinuous_dirtyMean[dataContinuous_dirtyMean.columns[np.where(newCoeff!=0)[0]]]
print ('Dirty Data: The Shape of New data is: ', newDirtyData.shape)

modelName='Dirty Data; Lasso Feature selection; Logistic with L1 Norm'
dirtyLasso_scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierLR_L1_poly)
dirtyLasso_scoreOBJ.setData(dataIN=newDirtyData, targetIN=targetBinomial_dirty)
dirtyLasso_scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

print ('Model (%s) : Performance Metric: \n'%modelName, modelScores[modelName])
print ('')

Clean Data: The Shape of New data is:  (1993, 34)
Model (Clean Data; Lasso Feature selection; Logistic with L1 Norm) : Performance Metric: 
 [{'accuracy': 0.85149999999999992}, {'precision': 0.8709122026050311}, {'recall': 0.88880138359548133}]

Dirty Data: The Shape of New data is:  (1994, 37)
Model (Dirty Data; Lasso Feature selection; Logistic with L1 Norm) : Performance Metric: 
 [{'accuracy': 0.84999999999999998}, {'precision': 0.88315316441279157}, {'recall': 0.88196204423858404}]



--> Poly Kernel SVM: Clean Data and Dirty Data:

In [40]:
from sklearn import svm

classifierSVM_poly = svm.SVC(kernel='poly', gamma=0.1, C=0.5, degree=3)


####### For Clean Data
newCoeff = classifierLS_clean.coef_.flatten()
newCleanData = dataContinuous[dataContinuous.columns[np.where(newCoeff!=0)[0]]]
print ('Clean Data: The Shape of New data is: ', newCleanData.shape)

modelName='Clean Data; Lasso Feature selection; Poly SVM'
cleanLasso_scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierSVM_poly)
cleanLasso_scoreOBJ.setData(dataIN=newCleanData, targetIN=targetBinomial)
cleanLasso_scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

print ('Model (%s) : Performance Metric: \n'%modelName, modelScores[modelName])
print ('')



#######  For Dirty Data
newCoeff = classifierLS_dirty.coef_.flatten()
newDirtyData = dataContinuous_dirtyMean[dataContinuous_dirtyMean.columns[np.where(newCoeff!=0)[0]]]
print ('Dirty Data: The Shape of New data is: ', newDirtyData.shape)

modelName='Dirty Data; Lasso Feature selection; Poly SVM'
dirtyLasso_scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierSVM_poly)
dirtyLasso_scoreOBJ.setData(dataIN=newDirtyData, targetIN=targetBinomial_dirty)
dirtyLasso_scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

print ('Model (%s) : Performance Metric: \n'%modelName, modelScores[modelName])
print ('')

Clean Data: The Shape of New data is:  (1993, 34)
Model (Clean Data; Lasso Feature selection; Poly SVM) : Performance Metric: 
 [{'accuracy': 0.83050000000000002}, {'precision': 0.84073677346707765}, {'recall': 0.89131720925893521}]

Dirty Data: The Shape of New data is:  (1994, 37)
Model (Dirty Data; Lasso Feature selection; Poly SVM) : Performance Metric: 
 [{'accuracy': 0.83699999999999997}, {'precision': 0.86248894734647141}, {'recall': 0.88574373106459969}]



## Part (b) :- Experiment with other learning methods such as polynomial or other kernels in SVM, decision forests, boosting, etc. and show your results. Make sure to explain clearly what you did.


Steps:

Here we build and evaluate three different models. 1) SVM with rbf Kernel, 2) Random Forest and 3) Adaboost. 

Step 1: First we define a range of coefficients for each parameter c and gamma for the rbf kernel. We run the SVM model for all different combination of c and gamma and then find the accuracy, precision and recall using a 10 fold cross validation

Step 2: Second we fit a random forest model for 500 decision trees for 10 fold cross validation and evaluate the model.

Step 3: Third, we fit a adaboost model with for 500 estimators for 10 fold cross validation and eveluate the models


Approach: Here we define a range of coefficients for each parameter "c" and "gamma", Using a grid like structure we fit a "rbf" kernel for 10-fold cross validation. "c" makes the trade of between the misclassification error and simpler model. A higher "c" would try to learn the training data and find more support vectors to support a very high degree of polynomial curve to fit all training points (non-regularized). "gamma" can be thought of as how much inverse of variance are we willing to trade of for the support vectors. A high value of "gamma" would result in the radius of the area of influence of the support vectors only including the support vector.

From the result we see that the best result is at [c= 1.0 and gamma = 0.01]	[acc=0.818322, precision=0.900897, recall=0.8624]. This indicates a small value of "gamma" which can be understood that the dataset doesn't have a very good decision split. The smaller "gamma" also indicates that the model is unable to find the complex structure in the data (even though it does, the model doesn't generalize well for the validation dataset), hence we can think of the model as close to a linear fit. Thats the rason that the evaluation metric of the "rbf" kernel is very close to "linear SVC".

#### i) What method gives the best results?

Solution: From the below evaluation score we can see that overall Random forest performs better than SVM (rbf) and adaboost. The Random forest in an ensemble technique that average the output from many decision tree for random samples.

SVM RBF: At c=1.0 and gamma=0.01: Performance Metric: {accuracy:0.818322,  precision:0.900897,  recall:0.8624}

Random Forest: num_trees=500 {'accuracy': 0.84199999999999997}, {'precision': 0.86346677944514327}, {'recall': 0.88492710688727683}

AdaBoost: estimators=500, {'accuracy': 0.81649999999999989}, {'precision': 0.84281571421416412}, {'recall': 0.85999477916895373} 

#### ii) What feature(s) seem to be most consistently predictive of high crime rates? How reliable is this conclusion?

Solution: From the best features we can see that features such as {PctKids2Par, TotalPctDiv, racePctWhite, PctFam2Par} are choosen by both the adaboost and randomforest model as important features. Additionally

SVM - RBF kernel: The solution is reliable in a sense that the dynamics of SVM is robust to overfitting. For classifying a new data point, despite the infinite dimensionality of the rbf kernel, we only have to go through support vectors, which are very less in number. The same has been processed through the crossvalidation. Moreover, The precision and recall scores are very high meaning that the algorithm is doing very good at identifying vilont crimes. Moreover cmpared to some of the above approaches the accuracy is also good enough.

Random Forest: Random Forest are very robust to overfitting as they fit many decision tree for randomly sampled data and average the knowledge of all the trees. Random forest were seen to be second best model just after logistic regression with Lasso feature selection. The performance of the random forest model is also consistent making is reliable for the dataset.

AdaBoost: 

--> SVM with RBF Kernel

In [40]:
from sklearn import svm

class SVM_model():

    def __init__(self, kernel='rbf'):
#         self.c_range = [0.01, 0.1, 1.0, 10.0, 100.0]#, 1000.0]            
#         self.gamma_range = [0.001, 0.01, 0.1, 1, 10.0, 100]  
        self.c_range = [0.01, 0.05, 0.1, 0.5, 1] #, 1000.0]            
        self.gamma_range = [0.01, 0.05, 0.1, 0.5, 1] 
        
        self.kernel=kernel

    def classify(self, trainData, trainLabels, validData):
        c_range = self.c_range
        gamma_range = self.gamma_range
        
        pred_dict = {}
        for (c,gamma) in itertools.product(c_range, gamma_range):
            string = "c" + str(c) + "_" + "gamma" + str(gamma)  
            clf = svm.SVC(kernel=self.kernel, C=c, gamma=gamma)
            classifier = clf.fit(trainData,trainLabels)
            pred_dict[string] = clf.predict(validData)
        return pred_dict



In [41]:
# Since the classes "HighCrime", "LowCrime" are unbalanced, We do stratifies Sampling and fetch 10-fold cross validation dataset.
import itertools
from sklearn.model_selection import StratifiedKFold

def buildModel(data, target):
    dataContinuous_NDarr = np.array(data, dtype='float64')
    targetBinomial_NDarr = np.array(target, dtype='float64')

    stN_Fold = StratifiedKFold(n_splits=10, random_state=675)  # We use the same seed, to get a rought comparison of the model with all other models.
    stN_Fold.get_n_splits(dataContinuous_NDarr, targetBinomial_NDarr)
#     print (stN_Fold)


    objSVM = SVM_model(kernel='rbf')
    outputEvaluationDict = {}
    validLabelDict = {}
    for foldNUM, (trainIndex, validIndex) in enumerate(stN_Fold.split(dataContinuous_NDarr, targetBinomial_NDarr)):
#         print("Train:", len(trainIndex), "CrossValid:", len(validIndex))
        if len(trainIndex)+len(validIndex) != len(dataContinuous):
            raise ValueError('The length of the samples doesnt match')


        outputEvaluationDict[foldNUM] = objSVM.classify(trainData=dataContinuous_NDarr[trainIndex], 
                                                        trainLabels=targetBinomial_NDarr[trainIndex], 
                                                        validData=dataContinuous_NDarr[validIndex])
        validLabelDict[foldNUM] = targetBinomial_NDarr[validIndex]
    return outputEvaluationDict, validLabelDict

In [42]:
# c_range = [0.01, 0.1, 1.0, 10.0, 100.0]#, 1000.0]            
# gamma_range = [0.001, 0.01, 0.1, 1, 10.0, 100] 

c_range = [0.01, 0.05, 0.1, 0.5, 1] #, 1000.0]            
gamma_range = [0.01, 0.05, 0.1, 0.5, 1] 

DF_PrecisionRecallFscore = pd.DataFrame(np.NaN, index=["c" + str(c) + "_" + "gamma" + str(gamma) for c, gamma in itertools.product(c_range, gamma_range)],
                                        columns=['avg-accuracy', 'avg-precision', 'avg-recall'])

outputEvaluationDict, validLabelDict = buildModel(dataContinuous, targetBinomial)

for foldNUM, c_gamma_prediction in  outputEvaluationDict.items():
#     print ('Running for cross validation fold : ', numFold)
    for c_gamma, prediction in c_gamma_prediction.items():
        if pd.isnull(DF_PrecisionRecallFscore.ix[c_gamma, 'avg-accuracy']):
            DF_PrecisionRecallFscore.ix[c_gamma, 'avg-accuracy'] = metrics.accuracy_score(validLabelDict[foldNUM], prediction)
        else:
            DF_PrecisionRecallFscore.ix[c_gamma, 'avg-accuracy'] = DF_PrecisionRecallFscore.ix[c_gamma, 'avg-accuracy'] + metrics.accuracy_score(validLabelDict[foldNUM], prediction)
            
        if pd.isnull(DF_PrecisionRecallFscore.ix[c_gamma, 'avg-precision']):
            DF_PrecisionRecallFscore.ix[c_gamma, 'avg-precision'] = metrics.precision_score(validLabelDict[foldNUM], prediction)
        else:
            DF_PrecisionRecallFscore.ix[c_gamma, 'avg-precision'] = DF_PrecisionRecallFscore.ix[c_gamma, 'avg-accuracy'] + metrics.precision_score(validLabelDict[foldNUM], prediction)
            
        if pd.isnull(DF_PrecisionRecallFscore.ix[c_gamma, 'avg-recall']):
            DF_PrecisionRecallFscore.ix[c_gamma, 'avg-recall'] = metrics.recall_score(validLabelDict[foldNUM], prediction)
        else:
            DF_PrecisionRecallFscore.ix[c_gamma, 'avg-recall'] = DF_PrecisionRecallFscore.ix[c_gamma, 'avg-recall'] + metrics.recall_score(validLabelDict[foldNUM], prediction)

DF_PrecisionRecallFscore/(foldNUM+1)

,avg-accuracy,avg-precision,avg-recall
c0.01_gamma0.01,0.627198,0.690013,1.0000
c0.01_gamma0.05,0.627198,0.690013,1.0000
c0.01_gamma0.1,0.654769,0.718544,0.9960
c0.01_gamma0.5,0.627198,0.690013,1.0000
c0.01_gamma1,0.627198,0.690013,1.0000
c0.05_gamma0.01,0.648756,0.712208,0.9976
c0.05_gamma0.05,0.789216,0.865033,0.8760
c0.05_gamma0.1,0.788206,0.864716,0.8680
c0.05_gamma0.5,0.749573,0.821524,0.9104
c0.05_gamma1,0.656796,0.722586,0.9776


##### Random Forests:

In [59]:
from sklearn.ensemble import RandomForestClassifier

classifierRF = RandomForestClassifier(n_estimators=500, criterion='gini')  # Entropy and Gini provides very similar evaluation results
classifierRF.fit(dataContinuous, targetBinomial)

modelName='Random Forest'
scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierRF)
scoreOBJ.setData(dataIN=dataContinuous, targetIN=targetBinomial)
scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

print (modelScores[modelName])

[{'accuracy': 0.84199999999999997}, {'precision': 0.86346677944514327}, {'recall': 0.88492710688727683}]


In [63]:
RFfeauturePredictivePower = pd.DataFrame(columns=['feature_name','absFeatureGain', 'featureGain'])

RFfeauturePredictivePower['feature_name'] = np.array(dataContinuous.columns)
RFfeauturePredictivePower['absFeatureGain'] = np.abs(classifierRF.feature_importances_)
RFfeauturePredictivePower['featureGain'] = classifierRF.feature_importances_
RFfeauturePredictivePower.sort_values('absFeatureGain', ascending=False).head(10)

,feature_name,absFeatureGain,featureGain
44,PctKids2Par,0.063366,0.063366
3,racePctWhite,0.061412,0.061412
50,PctIlleg,0.042549,0.042549
68,PctPersDenseHous,0.041143,0.041143
43,PctFam2Par,0.038669,0.038669
40,FemalePctDiv,0.034174,0.034174
41,TotalPctDiv,0.031145,0.031145
49,NumIlleg,0.028525,0.028525
2,racepctblack,0.024374,0.024374
45,PctYoungKids2Par,0.023780,0.023780


##### Ada Boost Classifier

In [68]:
from sklearn.ensemble import AdaBoostClassifier

classifierAB = AdaBoostClassifier(n_estimators=500, learning_rate=1)  # Entropy and Gini provides very similar evaluation results
classifierAB.fit(dataContinuous, targetBinomial)

modelName='Ada Boost Classifier'
scoreOBJ = ModelScoring(modelName = modelName, modelObj = classifierAB)
scoreOBJ.setData(dataIN=dataContinuous, targetIN=targetBinomial)
scoreOBJ.cvEvaluate(scoringMetric=['accuracy','precision','recall'])

print (modelScores[modelName])

[{'accuracy': 0.80999999999999994}, {'precision': 0.8353443501939537}, {'recall': 0.85848163563420066}]


In [65]:
ABfeauturePredictivePower = pd.DataFrame(columns=['feature_name','absFeatureGain', 'featureGain'])

ABfeauturePredictivePower['feature_name'] = np.array(dataContinuous.columns)
ABfeauturePredictivePower['absFeatureGain'] = np.abs(classifierAB.feature_importances_)
ABfeauturePredictivePower['featureGain'] = classifierAB.feature_importances_
ABfeauturePredictivePower.sort_values('absFeatureGain', ascending=False).head(10)

,feature_name,absFeatureGain,featureGain
88,MedOwnCostPctIncNoMtg,0.034,0.034
44,PctKids2Par,0.024,0.024
75,PctVacMore6Mos,0.024,0.024
23,indianPerCap,0.024,0.024
46,PctTeen2Par,0.022,0.022
15,pctWInvInc,0.020,0.020
41,TotalPctDiv,0.020,0.020
3,racePctWhite,0.020,0.020
64,PersPerOccupHous,0.020,0.020
22,blackPerCap,0.020,0.020


In [33]:
# One-hot encoding of the factor column state
dataMixed = pd.concat([dataContinuous, pd.get_dummies(crimeDF['state'])], axis=1)
dataMixed.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,46,47,48,49,50,51,53,54,55,56
0,0.01,0.61,0.21,0.83,0.02,0.01,0.41,0.49,0.26,0.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.01,0.41,0.55,0.57,0.01,0.00,0.47,0.45,0.31,0.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.03,0.34,0.86,0.30,0.04,0.01,0.41,0.42,0.27,0.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.01,0.38,0.35,0.71,0.04,0.01,0.39,0.46,0.31,0.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.04,0.37,0.32,0.70,0.21,0.02,1.00,1.00,1.00,0.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# Call the SVM rbf model with the state column:
outputEvaluationDict, validLabelDict = buildModel(dataMixed, targetBinomial)